#### Depencies

In [18]:
import geopy
import openml
import pandas as pd
import geopandas as gpd
import glob

from geopy.geocoders import Nominatim
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import os

# Data Profilling

## **An Exploratory Data Analysis of U.S Individual Donations for Political Campaigs**


---
A Comissão Federal de Eleições (FEC) é uma agência independente que visa mediar e regular o financiamente de campanhas presidencias dos EUA. Sob o pretexto de registrar cada doação dos cidadãos, criou-se a base que será aqui trabalhada. A partir das bibliotecas **Vega-Altair** e **Plotly**, explora-se a criação de visualizações para compreender e investigar o comportamento das contribuições políticas ao redor do país.


---



### **As variáveis da base**

#### Variáveis Preditivas

* **cmte_id**: ID do comite para o qual a contribuição foi feita. Cada comitê possui um identificador único. Neste dataset estão presentes 7112 comites diferentes.

* **amndt_ind**: Indicador de Emenda. Tal variável é usada para mostrar se uma transação ou um registro é uma emenda a um registro anteriormente apresentado, sendo classificada por uma das três opções:
  * N - Novo registro;

  * A - Emenda a um registro anterior;
  * C - Correção/alteração num registro.

* **rpt_tp**: Tipo de Relatório de contribuição do Comitê ao qual o contribuinte está associado. Seus tipos são comumente diferenciados pela sua recorrência(mensal, trimestral, pré-eleição e etc).

* **transaction_pgi**: Indicador referente a qual etapa do ciclo eleitoral a contribuição em questão está associada.
  *  G - General Election, fase final do ciclo, onde um candidato é eleito para o cargo
  *  P - Primary Election, refere-se às eleições que são internas aos partidos
  *  S - Special Election, eleições convocadas fora do ciclo eleitoral regular para preencher vagas inesperadas
  *  E - Election,
  *  C - Convenction, Convenções Partidárias, que podem ser usadas para selecionar candidatos ou tomar outras decisões internas do partido.
  * O - Other,

  * R - Runoff Election (Segundo Turno)

  * 0 - ??

* **image_num**: Identificador único associado à imagem digitalizada do relatório financeiro.

* **transaction_tp**
  * 10 - Contribuição Financeira direta de um indivíduo ou entidade para um comitê de campanha de um candidato
  * 11 - Contribuição de uma Tribo Nativa
  * 15 - Contribuição financeira direta feita por um indivíduo, sociedade ou LLC (Empresa de responsabilidade limitada) para comitês políticos tradicionais, excluindo Super PACs e Hybrid PACs.
  * 15C - Contribuição de candidatos
  * 15E - Contribuições Direcionadas feitas por indivíduos, sociedades ou LLC para comitês políticos tradicionais
  * 19 - Doação para comunicação eleitoral

  * 20Y - Reembolso de Fundos não Eleitorais. Devolução de fundos que foram originalmente atribuídos a atividades não eleitorais.
  * 21Y - Reembolso de uma Tribo Nativa
  * 22Y - Reembolso de uma contribuição de uma indivíduo, parceiros ou uma empresa de responsabilidade limitada
  * 24I - Contribuição Direcionada por cheque
  * 24T - Contribuição Direcionada utilizando fundos do tesouro do comitê intermediário

* **entity_tp**
  * IND - Individual
  * ORG - Organização
  * CCM - Comitê do Candidato
  * PAC - Comitê de Ação Política
  * CAN - Candidato
  * COM - Comitê

  * PTY - Comitê Oficial do Partido

* **name** - Nome do Contribuinte
* **city** - Cidade do Contribuinte

* **state** - Estado do Contribuinte

* **zip_code** - Código Postal do Contribuinte

* **employer** - Empregador do Contribuinte

* **ocuppation** - Emprego do Contribuinte

* **transaction_dt** - Data de transação das contribuições

* **other_id** - ID de identificação de uma terceira parte envolvida na transação

* **tran_id** - ID único da transação

* **file_num** - Número do arquivo

* **memo_cd** - Identificador associado a presença de um memorando na transação, o qual fornece um contexto adicional à transação

* **memo_text** - Texto do memorando

*  **sub_id** - Identificador de submissão

#### Variável Alvo
* **transaction_amt**: O valor da transação em dólares. Indica a quantidade de dinheiro envolvida em uma contrbuição individual para campanhas políticas.

Assim, o dataset é composto por 5 variáveis de Identificação única, 13 variáveis categóricas, uma variável de texto, uma variável de data e uma varável numérica (alvo)



In [2]:
dataset = openml.datasets.get_dataset(42080)
df, _, _, _ = dataset.get_data(dataset_format='dataframe')
path_shapefile = "cb_2018_us_state_500k\cb_2018_us_state_500k.shp"

state_boundaries = gpd.read_file(path_shapefile, columns=['STUSPS', 'NAME', 'geometry'])
state_boundaries.rename(columns={'STUSPS': 'state',
                           'Name': 'state_name'}, inplace=True)

df_EUA = state_boundaries.merge(df, on='state', how='inner')

In [3]:
df.shape

(3348209, 21)

In [4]:
# Número de categorias únicas das variáveis categóricas
categorical_cols = df.select_dtypes(include='object')
num_categories = categorical_cols.nunique().sort_values(ascending=False)
print(num_categories)

tran_id            2999272
name               1534998
employer            657449
occupation          145304
zip_code             86334
city                 27709
cmte_id               7112
other_id              2543
state                   67
rpt_tp                  26
transaction_tp          11
transaction_pgi          8
entity_tp                7
amndt_ind                3
memo_cd                  2
dtype: int64


# Preprocessing

In [5]:
#Percentual de Dados Faltantes
missing_percent = df.isna().sum().sort_values(ascending=False) / df.shape[0] * 100
missing_percent = missing_percent[missing_percent > 0]
print(missing_percent)

memo_cd            97.314325
other_id           92.992313
memo_text          87.438747
transaction_pgi    28.104966
employer           10.371216
occupation          5.546966
state               0.258556
city                0.064990
entity_tp           0.042948
transaction_dt      0.010573
tran_id             0.009766
name                0.004211
file_num            0.000030
dtype: float64


In [6]:
#Co-ocorrência das variáveis com valores faltantes significantes
significant_missing_columns = ['occupation', 'employer', 'transaction_pgi', 'memo_cd', 'memo_text', 'other_id']
co_occur = df[significant_missing_columns].isna().astype(int)
co_occur_agg = co_occur.groupby(significant_missing_columns).size().reset_index(name='count').sort_values(by='count', ascending=False)
co_occur_agg.head(15)

,occupation,employer,transaction_pgi,memo_cd,memo_text,other_id,count
7,0,0,0,1,1,1,1890857
15,0,0,1,1,1,1,683124
54,1,1,0,1,1,1,140372
5,0,0,0,1,0,1,93993
23,0,1,0,1,1,1,88439
4,0,0,0,1,0,0,85737
12,0,0,1,1,0,0,80218
13,0,0,1,1,0,1,60084
31,0,1,1,1,1,1,52315
1,0,0,0,0,0,1,47905


In [7]:
#Co-ocorrência das variáveis com dados faltantes que podem ser imputadas
significant_missing_columns = ['occupation', 'employer']
co_occur = df[significant_missing_columns].isna().astype(int)
co_occur_agg = co_occur.groupby(significant_missing_columns).size().reset_index(name='count').sort_values(by='count', ascending=False)
co_occur_agg.head(15)

,occupation,employer,count
0,0,0,2994621
3,1,1,179386
1,0,1,167864
2,1,0,6338


In [8]:
# Exclui-se as variáveis de indentificadores únicos
df.drop(columns=['name', 'sub_id', 'tran_id', 'image_num'], inplace=True)

# Exclui-se as variáveis 'memo_cd', 'other_id', 'memo_text'
df.drop(columns=['memo_cd', 'other_id', 'memo_text'], inplace=True)

colunas_com_poucos_dados_faltantes = ["state", "city", "entity_tp", "transaction_dt", "file_num"]
df = df.dropna(subset=colunas_com_poucos_dados_faltantes)
df['transaction_pgi'] = df['transaction_pgi'].fillna("Desconhecido")

## Geocoding from zip codes

In [ ]:
geolocator = Nominatim(user_agent="zip_code_locator")

# Cache para armazenar os resultados de ZIP codes já consultados
cache = {}

def get_location_from_zip(zip_code, max_retries=5):
    if zip_code in cache:
        print(f"zip code: {zip_code} já processado.")
        return zip_code, *cache[zip_code]
    
    # Implementar retentativas
    for attempt in range(max_retries):
        try:
            location = geolocator.geocode({"postalcode": zip_code, "country": "USA"})
            if location:
                result = (location.latitude, location.longitude, location.address.split(',')[1].strip())
                cache[zip_code] = result
                time.sleep(1.5)  # Reduz carga no serviço gratuito
                return zip_code, *result
        except Exception as e:
            print(f"Erro na tentativa {attempt + 1} para o ZIP code {zip_code}: {e}")
            time.sleep(2 ** attempt)  # Espera progressiva (backoff exponencial)
    
    # Retornar None se todas as tentativas falharem
    return zip_code, None, None, None


In [26]:
unique_zip_codes = df['zip_code'].unique()

chunk_size = 2000
output_dir = "geoprocessed_chunks"
os.makedirs(output_dir, exist_ok=True)

# Verificar quais chunks já foram processados
processed_chunks = {int(file.split('_')[-1].split('.')[0]) for file in os.listdir(output_dir) if file.endswith(".csv")}

for i in range(0, len(unique_zip_codes), chunk_size):
    chunk_index = i // chunk_size + 1  

    if chunk_index in processed_chunks:
        print(f"Chunk {chunk_index} já processado. Pulando...")
        continue

    chunk = unique_zip_codes[i:i+chunk_size]
    
    results = []
    for zip_code in tqdm(chunk, desc=f"Processando chunk {chunk_index}"):
        results.append(get_location_from_zip(zip_code))
    
    chunk_df = pd.DataFrame(results, columns=['zip_code', 'latitude', 'longitude', 'cidade'])
    
    chunk_file_name = os.path.join(output_dir, f"processed_zip_codes_chunk_{chunk_index}.csv")
    chunk_df.to_csv(chunk_file_name, index=False)
    print(f"Chunk {chunk_index} salvo com sucesso em {chunk_file_name}")
    
    

Chunk 1 já processado. Pulando...
Chunk 2 já processado. Pulando...
Chunk 3 já processado. Pulando...
Chunk 4 já processado. Pulando...


Processando chunk 5:   0%|          | 0/2000 [00:00<?, ?it/s]

zip code: 98204 já processado.
zip code: 33915 já processado.
zip code: 46055 já processado.
zip code: 46507 já processado.
zip code: 46540 já processado.
zip code: 46637 já processado.
zip code: 46614 já processado.
zip code: 46617 já processado.
zip code: 46176 já processado.
zip code: 89128 já processado.
zip code: 17837 já processado.
zip code: 37663 já processado.
zip code: 29206 já processado.
zip code: 44710 já processado.
zip code: 46163 já processado.
zip code: 32115 já processado.
zip code: 32169 já processado.
zip code: 32176 já processado.
zip code: 01741 já processado.
zip code: 49505 já processado.
zip code: 92782 já processado.
zip code: 30621 já processado.
zip code: 98684 já processado.
zip code: 37916 já processado.
zip code: 10467 já processado.
zip code: 30622 já processado.
zip code: 76104 já processado.


Processando chunk 5:   1%|▏         | 28/2000 [00:02<03:25,  9.58it/s]

zip code: 33446 já processado.
zip code: 33901 já processado.
zip code: 29506 já processado.
zip code: 80908 já processado.
zip code: 33130 já processado.
zip code: 16365 já processado.
zip code: 02215 já processado.
zip code: 98664 já processado.
zip code: 91786 já processado.
zip code: 22656 já processado.
zip code: 98195 já processado.


...

Erro na tentativa 2 para o ZIP code 29483.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?postalcode=29483.0&country=USA&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021B3E782040>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))
Erro na tentativa 3 para o ZIP code 29483.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?postalcode=29483.0&country=USA&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021B3E784250>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))
Erro na tentativa 4 para o ZIP code 29483.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?postalcode=29483.0&country=USA&format=json&limit=1 (Caused by NameResolutio

Processando chunk 6:  61%|██████    | 1213/2000 [10:26:40<109:04:39, 498.96s/it] 

Erro na tentativa 5 para o ZIP code 41143.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?postalcode=41143.0&country=USA&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021B3E7892E0>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))


Processando chunk 6:  72%|███████▏  | 1448/2000 [17:50:56<6:48:15, 44.38s/it]     


KeyboardInterrupt: 

## Merge final

In [19]:
output_dir = "geoprocessed_chunks"

all_chunk_files = glob.glob(os.path.join(output_dir, "processed_zip_codes_chunk_*.csv"))
processed_chunks_df = pd.concat([pd.read_csv(file) for file in all_chunk_files], ignore_index=True)
processed_chunks_df.shape

(6000, 4)

In [23]:
merge_1 = processed_chunks_df[['zip_code', 'latitude', 'longitude']]
amostra_teste = pd.merge(merge_1, df, on='zip_code', how='inner')
amostra_teste.head(5)

,zip_code,latitude,longitude,cmte_id,amndt_ind,rpt_tp,transaction_pgi,transaction_tp,entity_tp,city,state,employer,occupation,transaction_dt,transaction_amt,file_num
0,22303,38.792127,-77.081296,C00030718,N,M4,Desconhecido,15,IND,ALEXANDRIA,VA,COLDWELL BANKER RESIDENTIAL BK,REAL ESTATE BROKER,3092011.0,5.703782,724491.0
1,22303,38.792127,-77.081296,C00075820,N,M8,Desconhecido,15,IND,ALEXANDRIA,VA,INSTITUTE OF DEFENSE ANALYSES,INSTITUTE OF DEFENSE ANALYSES,7132011.0,6.907755,742738.0
2,22303,38.792127,-77.081296,C00287045,A,Q2,P,15,IND,ALEXANDRIA,VA,CAPTIAL STRATEGIES DC,PRINCIPAL,5312011.0,5.521461,743781.0
3,22303,38.792127,-77.081296,C00384818,N,M9,Desconhecido,15,IND,ALEXANDRIA,VA,"CAREMARK, L.L.C",SVP GOVNMT RELATIONS CVS CMK,8122011.0,6.030685,745409.0
4,22303,38.792127,-77.081296,C00193433,N,YE,Desconhecido,15,IND,ALEXANDRIA,VA,FINANCIAL ADVISOR,None,12142012.0,5.521461,848470.0


In [24]:
amostra_teste.to_csv("amostra.csv")